pandas (http://pandas.pydata.org/) is an open source, BSD-licensed library providing high-performance, easy-to-use data structures and data analysis tools for the Python programming language.

It's common to see pandas, numpy and matplotlib imported in the following way. We also have to specify that we would like generated images to be presented on this page. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

There are configuration options for Jupyter to do this automatically, this is useful if your notebooks will be used for similar types of data analysis.

First let's revisit the data we gathered earlier. We created a list of lists which pair up an IP address and how many times that IP address was seen in an nginx access log file.

In [2]:
ip_count = !cat access.log | awk '{print $1}' | sort | uniq -c | sort -rn
ip_count = [line.strip() for line in ip_count]
ip_count = [line.split() for line in ip_count][:10]
ip_count

[['206', '64.134.25.220'],
 ['138', '70.114.7.38'],
 ['115', '70.125.133.107'],
 ['109', '61.219.149.7'],
 ['93', '70.114.8.49'],
 ['80', '24.153.162.178'],
 ['50', '72.32.146.52'],
 ['47', '72.3.128.84'],
 ['46', '50.56.228.100'],
 ['46', '38.103.208.94']]

Now we want to take this data and have pandas be able to do something with it. We begin by creating a "DataFrame" from the 'ip_count' variable. DataFrames (DF from here on) are essentially spreadsheets that pandas can do some work on.

We can use the 'head' and 'tail' functions to get a quick peek at the DF without having to load the entire thing (especially useful if your DF is large).

In [3]:
df = pd.DataFrame(ip_count, columns=['count', 'IP'])
df.head()

,count,IP
0,206,64.134.25.220
1,138,70.114.7.38
2,115,70.125.133.107
3,109,61.219.149.7
4,93,70.114.8.49


In [4]:
df.tail()

,count,IP
5,80,24.153.162.178
6,50,72.32.146.52
7,47,72.3.128.84
8,46,50.56.228.100
9,46,38.103.208.94


***Note***: these dataframes are styled using html/css. Brandon Rhodes had an interesting presentation at PyCon 2015 which shows how to modify IPython's core css to style the DF: https://github.com/brandon-rhodes/pycon-pandas-tutorial I don't understand it enough to explain it so I won't be using it for this presentation

We can have pandas well us some information about the DF like what type of objects it's comprised of.

In [5]:
df.dtypes

count    object
IP       object
dtype: object

Uh-oh. We won't be able to do useful work unless pandas recognizes the 'count' column as a numeric type. 

#dtypes
from http://pandas.pydata.org/pandas-docs/stable/basics.html#dtypes

pandas understands several data types (dtypes). In this example we can see a couple of things: creating a DF from a dict and the various dtypes padas is aware of.

In [6]:
from pandas import Timestamp, Series
dft = pd.DataFrame(dict( A = np.random.rand(3),
                         B = 1,
                         C = 'foo',
                         D = Timestamp('20010102'),
                         E = Series([1.0]*3).astype('float32'),
                         F = False,
                         G = Series([1]*3,dtype='int8')))
dft.head()

,A,B,C,D,E,F,G
0,0.760740,1,foo,2001-01-02,1,False,1
1,0.164557,1,foo,2001-01-02,1,False,1
2,0.530205,1,foo,2001-01-02,1,False,1


Columns with string data are represented as the 'object' dtype (column 'C'). We'll need to coerce the data in our data to integers to work with them further.

In [7]:
dft.dtypes

A           float64
B             int64
C            object
D    datetime64[ns]
E           float32
F              bool
G              int8
dtype: object

To get column 'count' to integers we can 'apply' a function to a column:

In [8]:
df['count'] = df['count'].apply(int)
df.dtypes

count     int64
IP       object
dtype: object

We can also make use of lambda functions here:

In [9]:
df['count']

0    206
1    138
2    115
3    109
4     93
5     80
6     50
7     47
8     46
9     46
Name: count, dtype: int64

In [10]:
df['count'].apply(lambda x: x**2)

0    42436
1    19044
2    13225
3    11881
4     8649
5     6400
6     2500
7     2209
8     2116
9     2116
Name: count, dtype: int64

#Selecting

Now that our counts are integers we can use boolean operators to select rows from the DF

In [11]:
df['count'] > 100
#df[df['count'] > 100]

0     True
1     True
2     True
3     True
4    False
5    False
6    False
7    False
8    False
9    False
Name: count, dtype: bool

If we want to chain boolean checks we need to wrap them in parens since the '&' operator takes precedence over the '>' and '<' operators. If we were to try this without the parens pandas would complain that the "truth" of a series cannot be evaluated.

In [13]:
df[(df['count'] > 100) & (df['count'] < 200)]

,count,IP
1,138,70.114.7.38
2,115,70.125.133.107
3,109,61.219.149.7


# Shaping a DataFrame

Let's take a look at some example DFs 

In [37]:
df = pd.DataFrame(dict(A = [1, 2, 3],
                       B = [4, 5, 6],
                       C = [7, 8, 9]))
df

,A,B,C
0,1,4,7
1,2,5,8
2,3,6,9


In [42]:
df2 = pd.DataFrame(df.stack())
df2

0
0 A  1
  B  4
  C  7
1 A  2
  B  5
  C  8
2 A  3
  B  6
  C  9

# DB-like operations

http://pandas.pydata.org/pandas-docs/stable/merging.html#database-style-dataframe-joining-merging

With relational data we can perform operations as we would with a traditional RDBMS.

In pandas we can 'join' two DFs the way we would tables from a database. In this example We'll create a second DF that relates IP addresses to domain names

In [22]:
df2 = pd.DataFrame(dict(IP = ['64.134.25.220', '70.114.7.38', '70.125.133.107'], 
                        domain = ['example.com', 'example.net', 'example.org']))
df2

,IP,domain
0,64.134.25.220,example.com
1,70.114.7.38,example.net
2,70.125.133.107,example.org


By default pandas will perform an 'inner' join and pull data which exists in both DFs. The 'shape' of the resulting DF is (3,3)

In [27]:
df.merge(df2)

,count,IP,domain
0,206,64.134.25.220,example.com
1,138,70.114.7.38,example.net
2,115,70.125.133.107,example.org


But if we perform an 'outer' join the missing data will be filled in with NaNs

In [34]:
df.merge(df2, how='outer')

,count,IP,domain
0,206,64.134.25.220,example.com
1,138,70.114.7.38,example.net
2,115,70.125.133.107,example.org
3,109,61.219.149.7,NaN
4,93,70.114.8.49,NaN
5,80,24.153.162.178,NaN
6,50,72.32.146.52,NaN
7,47,72.3.128.84,NaN
8,46,50.56.228.100,NaN
9,46,38.103.208.94,NaN
